In [ ]:
!pip install qgrid
import qgrid
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler # Escalamiento estándar
from sklearn.model_selection import train_test_split   
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix 
import scipy as sc
import math
from numpy import random
from numpy import matlib
from scipy.spatial import distance
import matplotlib.pyplot as plt





In [ ]:
#Eliminamos las filas nulas
DatosConNulls = pd.read_csv("water_potability.csv", names = None, sep = ",")


In [ ]:
DatosConNulls.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

In [ ]:
#completar los valores en null con su media
df1=DatosConNulls[DatosConNulls['Potability']==1].copy()
df2=DatosConNulls[DatosConNulls['Potability']==0].copy()
df1['ph']=df1['ph'].replace(np.nan, df1['ph'].median())
df2['ph']=df2['ph'].replace(np.nan, df2['ph'].median())
df1['Sulfate']=df1['Sulfate'].replace(np.nan, df1['Sulfate'].median())
df2['Sulfate']=df2['Sulfate'].replace(np.nan, df2['Sulfate'].median())
df1['Trihalomethanes']=df1['Trihalomethanes'].replace(np.nan, df1['Trihalomethanes'].median())
df2['Trihalomethanes']=df2['Trihalomethanes'].replace(np.nan, df2['Trihalomethanes'].median())
Datos=pd.concat([df1,df2], axis=0, ignore_index=True)
Datos.isnull().sum()

ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64

In [ ]:
X = Datos.iloc[:,0:9] #Caracteristicas
Nombres = Datos.columns
Y = Datos['Potability']#clases
#Ya que la diferencia entre las escalas de los datos de X, se hará una normalización de los datos 
scaler = StandardScaler()
scaler.fit(X)
X_escalado = scaler.transform(X)


In [73]:
def completarTablaRedesNeuronales (i,tablaRedesNeuronales,EV,IC,F1,ICF1,PR):
  tablaRedesNeuronales.loc[i,"Porcentaje de reduccion"] = str(PR)
  tablaRedesNeuronales.loc[i,"Eficiencia en validacion"] = str(EV) # reemplazar los valores
  tablaRedesNeuronales.loc[i,"Intervalo de confianza de eficiencia"] = str(IC)
  tablaRedesNeuronales.loc[i,"f1-score"] = str(F1)
  tablaRedesNeuronales.loc[i,"Intervalo de confianza de f1-score"] = str(ICF1)
  return(tablaRedesNeuronales)

In [74]:
def completarTablaSoporteVectorial (i,tablaSoporteVectorial,EV,IC,PV,F1,ICF1,PR):
  tablaSoporteVectorial.loc[i,"Porcentaje de reduccion"] = str(PR)
  tablaSoporteVectorial.loc[i,"Eficiencia en validacion"] = str(EV)
  tablaSoporteVectorial.loc[i,"Intervalo de confianza de eficiencia"] = str(IC)
  tablaSoporteVectorial.loc[i, "% de Vectores de Soporte"] = str(PV)
  tablaSoporteVectorial.loc[i,"f1-score"] = str(F1)
  tablaSoporteVectorial.loc[i,"Intervalo de confianza de f1-score"] = str(ICF1)
  return(tablaSoporteVectorial)

In [92]:
def completarTablaBoostingTree (i,tablaBoostingTree,EV,IC,F1,ICF1,PR):
  tablaBoostingTree.loc[i,"Porcentaje de reduccion"] = str(PR)
  tablaBoostingTree.loc[i,"Eficiencia en validacion"] = str(EV) 
  tablaBoostingTree.loc[i,"Intervalo de confianza"] = str(IC)
  tablaBoostingTree.loc[i,"f1-score"] = str(F1)
  tablaBoostingTree.loc[i,"Intervalo de confianza de f1-score"] = str(ICF1)
  return(tablaBoostingTree)

# 1. Redes neuronales

In [ ]:
 #Integer encode
label_encoder = LabelEncoder()
integerY = label_encoder.fit_transform(Y)
print(integerY)

#Binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integerY = integerY.reshape(len(integerY), 1)
onehot_encoded = onehot_encoder.fit_transform(integerY)
print(onehot_encoded)

[1 1 1 ... 0 0 0]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [77]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
import scipy as sc
import math
from numpy import random
from numpy import matlib



tablaRedesNeuronales = pd.DataFrame({
    'N. de componentes' : pd.Series([1, 3, 5, 7, 9])})
Ncomp = np.array([1, 3, 5, 7, 9])

for i in range (5):

  Folds = 4
  random.seed(19680801)
  EficienciaTrain = np.zeros(Folds)
  EficienciaVal = np.zeros(Folds)
  f1 = np.zeros(Folds)
  skf = StratifiedKFold(n_splits=Folds)
  j = 0
  PR=(Ncomp[i]/9)*100
  for train, test in skf.split(X_escalado, integerY):
      Xtrain = X_escalado[train,:]
      Ytrain = integerY[train]
      Xtest = X_escalado[test,:]
      Ytest = integerY[test]
      Ytest=Ytest.reshape(len(Ytest))
      Ytrain=Ytrain.reshape(len(Ytrain))
      vector=(28,28)
      

      #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
      mlp = MLPClassifier(hidden_layer_sizes=vector, activation='identity', max_iter=400,verbose=True)
      mlp.out_activation_='softmax'


      pca = PCA(n_components= int(Ncomp[i]))
      X_train_pca = pca.fit_transform(X= Xtrain)
      X_test_pca = pca.transform(X= Xtest)
        
      mlp.fit(X=X_train_pca, y=Ytrain)
      Ytrain_pred = mlp.predict(X_train_pca)
      Yest = mlp.predict(X_test_pca)
     
      #Evaluamos las predicciones del modelo con los datos de test
      EficienciaTrain[j] = metrics.accuracy_score(Ytrain, Ytrain_pred)
      EficienciaVal[j] = metrics.accuracy_score(Ytest, Yest)
      f1[j]=metrics.f1_score(Ytest, Yest,average='weighted')
      j += 1
  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print("f1-score ",str(np.mean(f1))," +- ",np.std(f1))
  tablaRedesNeuronales = completarTablaRedesNeuronales(i,tablaRedesNeuronales,np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(f1),np.std(f1),PR)

Iteration 1, loss = 0.70905150
Iteration 2, loss = 0.67117229
Iteration 3, loss = 0.67018586
Iteration 4, loss = 0.66916167
Iteration 5, loss = 0.66892526
Iteration 6, loss = 0.66880459
Iteration 7, loss = 0.66889175
Iteration 8, loss = 0.66939481
Iteration 9, loss = 0.66870246
Iteration 10, loss = 0.66877628
Iteration 11, loss = 0.66922369
Iteration 12, loss = 0.66899438
Iteration 13, loss = 0.66900832
Iteration 14, loss = 0.66931773
Iteration 15, loss = 0.66908160
Iteration 16, loss = 0.66948035
Iteration 17, loss = 0.66914729
Iteration 18, loss = 0.66852729
Iteration 19, loss = 0.66873254
Iteration 20, loss = 0.66863653
Iteration 21, loss = 0.66962653
Iteration 22, loss = 0.67017822
Iteration 23, loss = 0.67005059
Iteration 24, loss = 0.66866610
Iteration 25, loss = 0.66925341
Iteration 26, loss = 0.66919096
Iteration 27, loss = 0.66869557
Iteration 28, loss = 0.66989900
Iteration 29, loss = 0.66933313
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. S

In [79]:
tablaRedesNeuronales

,Porcentaje de reduccion,Eficiencia en validacion,Intervalo de confianza de eficiencia,f1-score,Intervalo de confianza de f1-score
N. de componentes,,,,,
1,11.11111111111111,0.6098901098901099,0.0006105006105006638,0.4621012175471858,0.0007498897315181274
3,33.33333333333333,0.6095848595848595,0.0005287090377194837,0.4619574106212142,0.0006392943227449574
5,55.55555555555556,0.6092796092796093,0.0008633782432070539,0.4623657533052954,0.001074072828474141
7,77.77777777777779,0.6083638583638583,0.0015861271131583068,0.4613808706107788,0.0006126896616738307
9,100.0,0.6105006105006106,0.0014954149833841537,0.4645805283880841,0.003771691449024084


# 2. Máquinas de soporte vectorial

In [ ]:
import sklearn
from sklearn import preprocessing

#Se modifica cada etiqueta para que corresponda a un numero

dataSet=[]
label = np.unique(Y)
encoder=preprocessing.LabelEncoder()
encoder.fit(label)
data=encoder.transform(Y)
dataSet.append(data)
dataFrameY=pd.DataFrame(dataSet).values
dataFrameY=dataFrameY.T
dataFrameY = dataFrameY.astype(np.float)
dataFrameY=dataFrameY.ravel() 

In [83]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import qgrid


tablaSoporteVectorial = pd.DataFrame({
    'N. de componentes' : pd.Series([1, 3, 5, 7, 9])})
Ncomp = np.array([1, 3, 5, 7, 9])



for i in range (5):

  #Validamos el modelo
  Folds = 4
  random.seed(24524)
  EficienciaTrain = np.zeros(Folds)
  EficienciaVal = np.zeros(Folds)
  skf = StratifiedKFold(n_splits=Folds)
  j = 0
  porcentaje = np.zeros(4)
  PR=(Ncomp[i]/9)*100
  for train, test in skf.split(X_escalado, dataFrameY):
      Xtrain = X_escalado[train,:]
      Ytrain = dataFrameY[train]
      Xtest = X_escalado[test,:]
      Ytest = dataFrameY[test]
      
      #Normalizamos los datos
      scaler = preprocessing.StandardScaler().fit(Xtrain)
      Xtrain = scaler.transform(Xtrain)
      Xtest = scaler.transform(Xtest)
      
      modelo = SVC(kernel= 'rbf', C=10, gamma=0.1,decision_function_shape='ovr')
      
      pca = PCA(n_components= int(Ncomp[i]))
      X_train_pca = pca.fit_transform(X= Xtrain)
      X_test_pca = pca.transform(X= Xtest)
        
      modelo.fit(X=X_train_pca, y=Ytrain)
      Ytrain_pred = modelo.predict(X_train_pca)
      Yest = modelo.predict(X_test_pca)
      
      #Evaluamos las predicciones del modelo con los datos de test
      EficienciaTrain[j] = metrics.accuracy_score(Ytrain, Ytrain_pred)
      EficienciaVal[j] = metrics.accuracy_score(Ytest, Yest)
      f1[j]=metrics.f1_score(Ytest, Yest,average='weighted')
      porcentaje[j] =  len(modelo.support_vectors_) / len(Xtrain)
      
      j += 1
          
  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print("f1-score ",str(np.mean(f1))," +- ",np.std(f1))
  tablaSoporteVectorial=completarTablaSoporteVectorial(i,tablaSoporteVectorial,np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(porcentaje),np.mean(f1),np.std(f1),PR)

Eficiencia durante el entrenamiento = 0.6171143671143671+-0.009061473550588976
Eficiencia durante la validación = 0.6095848595848596+-0.005207180131023217
f1-score  0.46833141332731754  +-  0.009132436347130117
Eficiencia durante el entrenamiento = 0.6373626373626373+-0.010324502152190345
Eficiencia durante la validación = 0.6095848595848596+-0.003800335652621749
f1-score  0.48686919722559274  +-  0.007245508168959061
Eficiencia durante el entrenamiento = 0.6674806674806674+-0.006556371059660976
Eficiencia durante la validación = 0.6056166056166056+-0.004568568237819248
f1-score  0.5098024585596278  +-  0.01817518009745274
Eficiencia durante el entrenamiento = 0.733109483109483+-0.00569346147071132
Eficiencia durante la validación = 0.5927960927960928+-0.012496635830560883
f1-score  0.5402110840237375  +-  0.015851204312985326
Eficiencia durante el entrenamiento = 0.8263125763125763+-0.008172376628797304
Eficiencia durante la validación = 0.605006105006105+-0.036898689221639976
f1-scor

In [84]:
tablaSoporteVectorial

,N. de componentes,Porcentaje de reduccion,Eficiencia en validacion,Intervalo de confianza de eficiencia,% de Vectores de Soporte,f1-score,Intervalo de confianza de f1-score
0,1,11.11111111111111,0.6095848595848596,0.005207180131023217,0.7868335368335369,0.46833141332731754,0.009132436347130117
1,3,33.33333333333333,0.6095848595848596,0.003800335652621749,0.7602767602767604,0.48686919722559274,0.007245508168959061
2,5,55.55555555555556,0.6056166056166056,0.004568568237819248,0.7582417582417582,0.5098024585596278,0.01817518009745274
3,7,77.77777777777779,0.5927960927960928,0.012496635830560883,0.7421652421652423,0.5402110840237375,0.015851204312985326
4,9,100.0,0.605006105006105,0.036898689221639976,0.6903744403744403,0.5859246133455577,0.035034982603876705


# 3. Gradient boosting tree

In [107]:
tablaBoostingTree = pd.DataFrame({
    'N. de componentes' : pd.Series([1, 3, 5, 7, 9])})
Ncomp = np.array([1, 3, 5, 7, 9])


for i in range(5):
  clf =  GradientBoostingClassifier(n_estimators=300, learning_rate=0.3	,max_depth=5, random_state=0)
  Folds = 4
  EficienciaTrain = np.zeros(Folds)
  EficienciaVal = np.zeros(Folds)
  f1 = np.zeros(Folds)
  skf = StratifiedKFold(n_splits=Folds)
  j = 0
  PR=(Ncomp[i]/9)*100
  for train, test in skf.split(X_escalado, integerY):
      Xtrain = X_escalado[train,:]
      Ytrain = integerY[train]
      Xtest = X_escalado[test,:]
      Ytest = integerY[test]
      Ytest=Ytest.reshape(len(Ytest))
      Ytrain=Ytrain.reshape(len(Ytrain))

      pca = PCA(n_components= int(Ncomp[i]))
      X_train_pca = pca.fit_transform(X= Xtrain)
      X_test_pca = pca.transform(X= Xtest)
      clf.fit(X=X_train_pca, y=Ytrain)
      Ytrain_pred = clf.predict(X_train_pca)
      Yest = clf.predict(X_test_pca)

      #Evaluamos las predicciones del modelo con los datos de test
      EficienciaTrain[j] = metrics.accuracy_score(Ytrain, Ytrain_pred)
      EficienciaVal[j] = metrics.accuracy_score(Ytest, Yest)
      f1[j]=metrics.f1_score(Ytest, Yest,average='weighted')
      j += 1
  print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
  print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
  print("f1-score ",str(np.mean(f1))," +- ",np.std(f1))
  tablaBoostingTree=completarTablaBoostingTree(i,tablaBoostingTree,np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(f1),np.std(f1),PR)

Eficiencia durante el entrenamiento = 0.9824989824989825+-0.003935506635289743
Eficiencia durante la validación = 0.5317460317460317+-0.021816322814658912
f1-score  0.5296610080631388  +-  0.021121511339702057
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.5656288156288156+-0.007930635565791534
f1-score  0.550375828557387  +-  0.005118558648353892
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.5589133089133089+-0.013104449287952187
f1-score  0.5421379662352137  +-  0.009375768364149314
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.5641025641025641+-0.0229405301929865
f1-score  0.5466750291969696  +-  0.021512012974518498
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.5695970695970696+-0.03842275696595741
f1-score  0.5590752635743864  +-  0.03488249656114261


In [103]:
tablaBoostingTree

,N. de componentes,Porcentaje de reduccion,Eficiencia en validacion,Intervalo de confianza,f1-score,Intervalo de confianza de f1-score
0,1,11.11111111111111,0.5317460317460317,0.021816322814658912,0.5296610080631388,0.021121511339702057
1,3,33.33333333333333,0.5656288156288156,0.007930635565791534,0.550375828557387,0.005118558648353892
2,5,55.55555555555556,0.5589133089133089,0.013104449287952187,0.5421379662352137,0.009375768364149314
3,7,77.77777777777779,0.5641025641025641,0.0229405301929865,0.5466750291969696,0.021512012974518498
4,9,100.0,0.5695970695970696,0.03842275696595741,0.5590752635743864,0.03488249656114261
